### Generate Support pickle files for deployment

In [1]:
# Load libraries

import pandas as pd
import pickle
import numpy as np

In [2]:
product_mappings = pd.read_csv("products.csv")

In [3]:
product_mappings

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5
49684,49685,En Croute Roast Hazelnut Cranberry,42,1
49685,49686,Artisan Baguette,112,3
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8


#### Reduce Size of products.csv

In [4]:
product_mappings = product_mappings[['product_id','product_name']]

In [5]:
product_mappings.to_pickle("product_mappings.pkl")

#### Get top 10 products of each hour of each day of week

In [6]:
aisles = pd.read_csv('aisles.csv')
departments = pd.read_csv('departments.csv')
prior_order = pd.read_csv('order_products__prior.csv')
train_order = pd.read_csv('order_products__train.csv')
orders = pd.read_csv('orders.csv')
products = pd.read_csv('products.csv')

#imputation
orders.fillna(value = 0, inplace = True)


prior_train_orders = prior_order.sort_values(by=['order_id'])
prior_train_orders = pd.merge(left = prior_train_orders, right = products,
                             left_on='product_id', right_on='product_id').sort_values(by=['order_id']).reset_index(drop=True)
prior_train_orders = pd.merge(left = prior_train_orders, right = aisles,
                             left_on='aisle_id', right_on='aisle_id').sort_values(by=['order_id']).reset_index(drop=True)
prior_train_orders = pd.merge(left = prior_train_orders, right = departments,
                             left_on='department_id', right_on='department_id').sort_values(by=['order_id']).reset_index(drop=True)
prior_train_orders = pd.merge(left = prior_train_orders, right = orders,
                             left_on='order_id', right_on='order_id').sort_values(by=['order_id']).reset_index(drop=True)

col_order = ['user_id',
 'order_id',
 'product_id',
  'aisle_id',
 'department_id',
 'add_to_cart_order',
 'reordered',
 'product_name',
 'aisle',
 'department',
 'eval_set',
 'order_number',
 'order_dow',
 'order_hour_of_day',
 'days_since_prior_order']

prior_train_orders = prior_train_orders[col_order]
print("Shape of Prior_train_orders :", prior_train_orders.shape)
prior_train_orders.head()

Shape of Prior_train_orders : (32434489, 15)


,user_id,order_id,product_id,aisle_id,department_id,add_to_cart_order,reordered,product_name,aisle,department,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,202279,2,33120,86,16,1,1,Organic Egg Whites,eggs,dairy eggs,prior,3,5,9,8.0
1,202279,2,1819,88,13,8,1,All Natural No Stir Creamy Almond Butter,spreads,pantry,prior,3,5,9,8.0
2,202279,2,45918,19,13,4,1,Coconut Butter,oils vinegars,pantry,prior,3,5,9,8.0
3,202279,2,9327,104,13,3,0,Garlic Powder,spices seasonings,pantry,prior,3,5,9,8.0
4,202279,2,17794,83,4,6,1,Carrots,fresh vegetables,produce,prior,3,5,9,8.0


In [7]:
x = prior_train_orders.groupby(['order_dow','order_hour_of_day','product_name'])['product_name'].size().reset_index(name = 'count').sort_values(
   by=['order_dow','order_hour_of_day','count'], ascending = False)
top10 = x.groupby(['order_dow','order_hour_of_day']).head(10).reset_index(drop = True)

In [9]:
top10[(top10['order_dow']==5) & (top10['order_hour_of_day'] ==5)]

,order_dow,order_hour_of_day,product_name,count
420,5,5,Banana,219
421,5,5,Bag of Organic Bananas,136
422,5,5,Organic Strawberries,125
423,5,5,Organic Hass Avocado,103
424,5,5,Organic Baby Spinach,101
425,5,5,Strawberries,81
426,5,5,Limes,68
427,5,5,Organic Avocado,67
428,5,5,Large Lemon,64
429,5,5,Organic Raspberries,58


In [10]:
top10.to_pickle("top10_products.pkl")

#### Add User's last purchase date 

In [46]:
#assuming last order placed by every user was yesterday 

user_last_purchase = pd.DataFrame(columns = ['user_id','date'])
user_last_purchase['user_id'] = orders['user_id'].unique()
user_last_purchase['date'] = '2021-03-21'

In [47]:
user_last_purchase.to_pickle("user_last_purchase.pkl")